In [1]:
import os
import pandas as pd
import numpy as np

In [3]:
def load_and_preprocess_csv(csv_path, segment_length=500):
    """
    Carga un archivo CSV, inserta una fila vacía y utiliza automáticamente la primera fila del archivo.
    """
    df = pd.read_csv(csv_path)
    
    # Guardar la primera fila del archivo CSV antes de insertar la fila vacía
    first_row_values = df.iloc[0].tolist()
    
    # Insertar una fila vacía al principio
    df.loc[-1] = [np.nan] * len(df.columns)  # Crear una fila vacía
    df.index = df.index + 1  # Desplazar los índices hacia abajo
    df = df.sort_index()  # Ordenar el índice para que la fila vacía quede en la parte superior

    # Asignar los valores originales de la primera fila a la nueva primera fila
    df.iloc[0] = first_row_values

    # Definir los nombres de las columnas si es necesario
    column_names = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8']
    df.columns = column_names[:len(df.columns)]  # Asignar nombres de columnas dinámicamente
    
    # Dividir el DataFrame en segmentos
    num_segments = len(df) // segment_length  # Número total de segmentos completos
    segments = [df.iloc[i * segment_length: (i + 1) * segment_length] for i in range(num_segments)]
    
    return segments

def calculate_statistics(df):
    """
    Calcula las estadísticas para un DataFrame y devuelve un nuevo DataFrame con los resultados.
    """
    stats = {}
    
    # Iterar por cada columna y calcular las estadísticas
    for column in df.columns:
        stats[f'{column}_mean'] = df[column].mean()
        stats[f'{column}_median'] = df[column].median()
        stats[f'{column}_std'] = df[column].std()
        stats[f'{column}_var'] = df[column].var()
        stats[f'{column}_cv'] = df[column].std() / df[column].mean()
        stats[f'{column}_min'] = df[column].min()
        stats[f'{column}_max'] = df[column].max()
        stats[f'{column}_IQR'] = df[column].quantile(0.75) - df[column].quantile(0.25)

    # Convertir el diccionario en un DataFrame de una sola fila
    return pd.DataFrame(stats, index=[0])

def add_custom_columns(df, col_names, values):
    """
    Añade columnas personalizadas al DataFrame.
    """
    for col_name, value in zip(col_names, values):
        df[col_name] = value
    return df

def process_csvs_in_directory(base_dir, segment_length=500):
    """
    Recorre la estructura de directorios, procesa los CSVs y asigna columnas personalizadas.
    """
    processed_data = []

    for root, dirs, files in os.walk(base_dir):
        # Identificar archivos CSV
        csv_files = [f for f in files if f.endswith('.csv')]
        
        for csv_file in csv_files:
            csv_path = os.path.join(root, csv_file)
            
            # Extraer los valores de las columnas personalizadas del nombre de archivo y ruta
            file_name = os.path.splitext(csv_file)[0]
            folder_name = os.path.basename(root)
            parent_folder = os.path.basename(os.path.dirname(root))
            
            custom_column_names = ['Hz', 'medida', 'Tipo']
            custom_values = [file_name, folder_name, parent_folder]
            
            # Procesar el archivo CSV
            segments = load_and_preprocess_csv(csv_path, segment_length)
            
            for segment in segments:
                # Calcular estadísticas
                stats_df = calculate_statistics(segment)
                # Añadir columnas personalizadas
                stats_df = add_custom_columns(stats_df, custom_column_names, custom_values)
                # Almacenar el DataFrame procesado
                processed_data.append(stats_df)

    return processed_data

# Ejemplo de uso:

# Ruta base del directorio que contiene las carpetas y CSVs
base_dir = '../bearing_fault'

# Procesar todos los CSVs en la estructura de directorios
processed_dfs = process_csvs_in_directory(base_dir, segment_length=500)

# Mostrar el primer DataFrame procesado
if processed_dfs:
    print(processed_dfs[0])


    S1_mean  S1_median    S1_std    S1_var     S1_cv   S1_min  S1_max  \
0  0.286609   -0.45415  1.760644  3.099869  6.143024 -0.74812  4.8012   

    S1_IQR   S2_mean  S2_median  ...  S8_median    S8_std   S8_var     S8_cv  \
0  0.18663  0.021666   0.031395  ...  -0.023922  0.080811  0.00653 -7.557894   

    S8_min   S8_max    S8_IQR      Hz  medida                     Tipo  
0 -0.15725  0.34543  0.118908  12.288   0.5mm  horizontal-misalignment  

[1 rows x 67 columns]


In [8]:
processed_dfs

[    S1_mean  S1_median    S1_std    S1_var     S1_cv   S1_min  S1_max  \
 0  0.286609   -0.45415  1.760644  3.099869  6.143024 -0.74812  4.8012   
 
     S1_IQR   S2_mean  S2_median  ...  S8_median    S8_std   S8_var     S8_cv  \
 0  0.18663  0.021666   0.031395  ...  -0.023922  0.080811  0.00653 -7.557894   
 
     S8_min   S8_max    S8_IQR      Hz  medida                     Tipo  
 0 -0.15725  0.34543  0.118908  12.288   0.5mm  horizontal-misalignment  
 
 [1 rows x 67 columns],
    S1_mean  S1_median    S1_std    S1_var    S1_cv   S1_min  S1_max    S1_IQR  \
 0  3.45561    4.51815  2.094236  4.385825  0.60604 -0.64089  4.6356  0.092075   
 
     S2_mean  S2_median  ...  S8_median    S8_std    S8_var     S8_cv   S8_min  \
 0 -0.477666   -0.43153  ...   0.025264  0.115949  0.013444  3.698598 -0.16506   
 
     S8_max    S8_IQR      Hz  medida                     Tipo  
 0  0.34609  0.206634  12.288   0.5mm  horizontal-misalignment  
 
 [1 rows x 67 columns],
    S1_mean  S1_median  

In [1]:
import os
import pandas as pd
import numpy as np

def load_and_preprocess_csv(csv_path, segment_length=500):
    """
    Carga un archivo CSV, inserta una fila vacía y utiliza automáticamente la primera fila del archivo.
    """
    df = pd.read_csv(csv_path)
    
    # Guardar la primera fila del archivo CSV antes de insertar la fila vacía
    first_row_values = df.iloc[0].tolist()
    
    # Insertar una fila vacía al principio
    df.loc[-1] = [np.nan] * len(df.columns)  # Crear una fila vacía
    df.index = df.index + 1  # Desplazar los índices hacia abajo
    df = df.sort_index()  # Ordenar el índice para que la fila vacía quede en la parte superior

    # Asignar los valores originales de la primera fila a la nueva primera fila
    df.iloc[0] = first_row_values

    # Definir los nombres de las columnas si es necesario
    column_names = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8']
    df.columns = column_names[:len(df.columns)]  # Asignar nombres de columnas dinámicamente
    
    # Dividir el DataFrame en segmentos
    num_segments = len(df) // segment_length  # Número total de segmentos completos
    segments = [df.iloc[i * segment_length: (i + 1) * segment_length] for i in range(num_segments)]
    
    return segments

def calculate_statistics(df):
    """
    Calcula las estadísticas para un DataFrame y devuelve un nuevo DataFrame con los resultados.
    """
    stats = {}
    
    # Iterar por cada columna y calcular las estadísticas
    for column in df.columns:
        stats[f'{column}_mean'] = df[column].mean()
        stats[f'{column}_median'] = df[column].median()
        stats[f'{column}_std'] = df[column].std()
        stats[f'{column}_var'] = df[column].var()
        stats[f'{column}_cv'] = df[column].std() / df[column].mean()
        stats[f'{column}_min'] = df[column].min()
        stats[f'{column}_max'] = df[column].max()
        stats[f'{column}_IQR'] = df[column].quantile(0.75) - df[column].quantile(0.25)

    # Convertir el diccionario en un DataFrame de una sola fila
    return pd.DataFrame(stats, index=[0])

def add_custom_columns(df, col_names, values):
    """
    Añade columnas personalizadas al DataFrame.
    """
    for col_name, value in zip(col_names, values):
        df[col_name] = value
    return df

def process_csvs_in_directory(base_dir, segment_length=500):
    """
    Recorre la estructura de directorios, procesa los CSVs y asigna columnas personalizadas.
    """
    processed_data = []

    for root, dirs, files in os.walk(base_dir):
        # Identificar archivos CSV
        csv_files = [f for f in files if f.endswith('.csv')]
        
        for csv_file in csv_files:
            csv_path = os.path.join(root, csv_file)
            
            # Extraer los valores de las columnas personalizadas del nombre de archivo y ruta
            file_name = os.path.splitext(csv_file)[0]
            folder_name = os.path.basename(root)
            parent_folder = os.path.basename(os.path.dirname(root))
            
            custom_column_names = ['Hz', 'medida', 'Tipo']
            custom_values = [file_name, folder_name, parent_folder]
            
            # Procesar el archivo CSV
            segments = load_and_preprocess_csv(csv_path, segment_length)
            
            for segment in segments:
                # Calcular estadísticas
                stats_df = calculate_statistics(segment)
                # Añadir columnas personalizadas
                stats_df = add_custom_columns(stats_df, custom_column_names, custom_values)
                # Almacenar el DataFrame procesado
                processed_data.append(stats_df)

    # Concatenar todos los DataFrames en uno solo
    if processed_data:
        return pd.concat(processed_data, ignore_index=True)
    else:
        return pd.DataFrame()  # Retornar un DataFrame vacío si no hay datos procesados



In [1]:
#USO
base_dir = '../bearing_fault'

# Procesar todos los CSVs en la estructura de directorios y concatenar los resultados
final_df = process_csvs_in_directory(base_dir, segment_length=500)

# Guardar el DataFrame final en un archivo CSV
final_df.to_csv('processed_data.csv', index=False)

final_df


NameError: name 'process_csvs_in_directory' is not defined